In [1]:
import pandas as pd
import numpy as np
import re
import evaluate
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [2]:
df = pd.read_csv("../data/train.csv")
df.head()

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1


In [3]:
df = df.drop(columns=["row_id"])

In [27]:
df = df.rename(columns={"rule_violation": "label"})

In [4]:
features = df.columns.tolist()
features.remove("rule_violation")
features

['body',
 'rule',
 'subreddit',
 'positive_example_1',
 'positive_example_2',
 'negative_example_1',
 'negative_example_2']

In [5]:
df['rule_violation'].value_counts()

rule_violation
1    1031
0     998
Name: count, dtype: int64

### Splitting data into train and test

In [30]:
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

In [31]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [32]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [33]:
def tokenize_function(examples):
    texts = [" ".join([examples[col][i] for col in features]) for i in range(len(examples[features[0]]))]
    return tokenizer(texts, padding="max_length", truncation=True)

In [34]:
# Apply the tokenization to our datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1623 [00:00<?, ? examples/s]

Map:   0%|          | 0/406 [00:00<?, ? examples/s]

In [35]:
tokenized_train_dataset.shape

(1623, 11)

In [36]:
tokenized_train_dataset.column_names

['body',
 'rule',
 'subreddit',
 'positive_example_1',
 'positive_example_2',
 'negative_example_1',
 'negative_example_2',
 'label',
 '__index_level_0__',
 'input_ids',
 'attention_mask']

In [37]:
type(tokenized_train_dataset)

datasets.arrow_dataset.Dataset

In [38]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
metric = evaluate.load("accuracy")

In [40]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### Setting Hyperparameters and Training the Model

In [41]:
training_args = TrainingArguments(
    output_dir="comment_violation_classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True
)

In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\HP\AppData\Local\Temp\ipykernel_23332\3788975122.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### Start the training process

In [43]:
trainer.train()

d:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.585114,0.709360
2,No log,0.548802,0.701970


d:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=204, training_loss=0.6042084787406173, metrics={'train_runtime': 3184.544, 'train_samples_per_second': 1.019, 'train_steps_per_second': 0.064, 'total_flos': 429989176037376.0, 'train_loss': 0.6042084787406173, 'epoch': 2.0})

In [44]:
# Save the final model
trainer.save_model("comment_violation_classifier_model")

### Evaluating the model

In [45]:
# Evaluate on the training set
print("Evaluating on the training set...")
train_predictions = trainer.predict(tokenized_train_dataset)
train_preds = np.argmax(train_predictions.predictions, axis=-1)
print(classification_report(train_df['label'], train_preds))

# Evaluate on the testing set
print("Evaluating on the testing set...")
test_predictions = trainer.predict(tokenized_test_dataset)
test_preds = np.argmax(test_predictions.predictions, axis=-1)
print(classification_report(test_df['label'], test_preds))

Evaluating on the training set...


d:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


              precision    recall  f1-score   support

           0       0.82      0.67      0.74       798
           1       0.73      0.86      0.79       825

    accuracy                           0.76      1623
   macro avg       0.77      0.76      0.76      1623
weighted avg       0.77      0.76      0.76      1623

Evaluating on the testing set...


d:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


              precision    recall  f1-score   support

           0       0.74      0.61      0.67       200
           1       0.68      0.79      0.73       206

    accuracy                           0.70       406
   macro avg       0.71      0.70      0.70       406
weighted avg       0.71      0.70      0.70       406



### Predict test data

In [46]:
# Load the test data
new_test_df = pd.read_csv("../data/test.csv")
new_test_df.head()

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2
0,2029,NEW RAP GROUP 17. CHECK US OUT https://soundcl...,"No Advertising: Spam, referral links, unsolici...",hiphopheads,"Hey, guys, just wanted to drop in and invite y...",Cum Swallowing Hottie Katrina Kaif Cartoon Xvi...,SD Stream Eng - [Chelsea TV USA](http://soccer...,HD Streams: |[ENG HD Stoke vs Manchester Unite...
1,2030,Make your life comfortable. Get up to 15% Disc...,No legal advice: Do not offer or request legal...,AskReddit,Get a lawyer and get the security camera foota...,That isn't drastic. You tried reaching out to ...,So what are you going to do with the insurance...,It's just for Austria & Germany. If you still ...
2,2031,Kickin' ass and selling underwear!\nJust made ...,"No Advertising: Spam, referral links, unsolici...",gonewild,Good story my friend. Check out my blog at ht...,If you know what exactly you need then you don...,CENTIPEDES\n\nSOME BASED PATRIOTS HAVE CREATED...,[So great! Thanks for sharing.](http://www.che...
3,2032,watch hooters best therein http://clickan...,"No Advertising: Spam, referral links, unsolici...",personalfinance,"Earn 50,000 bonus points with Chase Sapphire P...","Cool, front page! I made this print along with...",[Full HD Movie Online Free](http://www.flickma...,* Karambit Black Pearl\n* 0.02137822 Float (un...
4,2033,bitches for free at this point show all h...,"No Advertising: Spam, referral links, unsolici...",Showerthoughts,code free tyrande --->>> [Imgur](http://i.imgu...,My trade link\nhttps://steamcommunity.com/trad...,**HD** [ mio Stadium 102 HD](http://www.genti....,Infographics is an incredible method for showi...


In [47]:
new_test_df = new_test_df.drop(columns=["row_id"])

# Tokenize the new test data
def tokenize_new_test(examples):
    texts = [" ".join([examples[col][i] for col in features]) for i in range(len(examples[features[0]]))]
    return tokenizer(texts, padding="max_length", truncation=True)

new_test_dataset = Dataset.from_pandas(new_test_df)
tokenized_new_test_dataset = new_test_dataset.map(tokenize_new_test, batched=True)

# Predict using the trained model
new_test_predictions = trainer.predict(tokenized_new_test_dataset)
new_test_preds = np.argmax(new_test_predictions.predictions, axis=-1)

# If you want to save predictions to a CSV
new_test_df["predicted_label"] = new_test_preds
new_test_df.to_csv("test_predictions.csv", index=False)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

d:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
